In [ ]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import json

#! pip install ibm_watson
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 \
    import Features, EmotionOptions

from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/PSVII")

# Load data for testing (assuming CSV file format for example)
df = pd.read_csv('IBM_GoEmotions.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Authenticate to IBM Watson
authenticator = IAMAuthenticator('')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2022-04-07',
    authenticator=authenticator
)
natural_language_understanding.set_service_url('')

def analyze_emotion(text):
    try:
        response = natural_language_understanding.analyze(
            text=text,
            features=Features(emotion=EmotionOptions(document=True))
        ).get_result()
        # Extracting emotion scores from the response
        emotions = response['emotion']['document']['emotion']
        # Return a Series for easy DataFrame concatenation
        return pd.Series(emotions)
    except Exception as e:
        print(f"Error processing text: {e}")
        # Return NaNs in case of an error
        return pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan], index=['sadness', 'joy', 'fear', 'disgust', 'anger'])

# Apply the function and join the results as new columns
emotion_df = df['clean_text'].apply(analyze_emotion)
emotion_df.columns = ['predicted_sadness', 'predicted_joy', 'predicted_fear', 'predicted_disgust', 'predicted_anger']

# Concatenate the original DataFrame with the new emotion columns
df = pd.concat([df, emotion_df], axis=1)

# Saving the DataFrame to an Excel file
# df.to_excel('IBM-LITE_Output.xlsx', index=False)
# print("The file has been successfully saved")